## Introduction

Aim of this notebook is to calibrate the Heston model using deep learning technique introduced by *Horvath et al.* in *Deep Learning Volatility*


The notebook will replicate the model given in the paper and use SPX option prices from Yahoo Finance for testing. Deep learning training data will be generated synthetically.


 References: https://arxiv.org/abs/1901.09647

In [1]:
# import libaries

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.functional import mse_loss, l1_loss


from torch.utils.data import TensorDataset, DataLoader, random_split 
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import itertools
import time 
import pickle
import os


from sklearn.metrics import mean_absolute_error

import yfinance as yf
import datetime

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Model will run/train on:", device)

Model will run/train on: cuda


In [2]:
# seeds to allow reproducibility 

seed_val = 0

np.random.seed(0)
torch.manual_seed(0)

gen1 = torch.Generator().manual_seed(0)


torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)

In [ ]:
# Model paramters

batch_size = 32
epochs = 200

run_training = True

generate_training_data = False
 
strikes = np.arange(5800,6900, step = 100).tolist()

maturities = np.arange(0.048,0.11, step = 1/252).round(3).tolist()

maturities_year = np.arange(0.0,1, step = 1/252).round(3).tolist()

maturities_idx = [12,14,16,18,20,22,24,26]

In [4]:
from nelson_siegel_svensson import NelsonSiegelSvenssonCurve
from nelson_siegel_svensson.calibrate import calibrate_nss_ols

yield_maturities = np.array([1/12, 2/12, 3/12,4/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
yields = np.array([4.92,4.82,4.73,4.66,4.47,4.24,4.02,3.95,3.98,4.07,4.19,4.54,4.49]).astype(float)/100

#NSS model calibrate
curve_fit, status = calibrate_nss_ols(yield_maturities,yields)

#Can use the fitted curve to get rate at any tenor point
curve_fit

NelsonSiegelSvenssonCurve(beta0=0.04563032361939878, beta1=0.0036849763147745952, beta2=-0.036081271617350925, beta3=0.01048653576721284, tau1=2.0, tau2=5.0)

In [5]:
def generate_heston_paths(S0, V0, mu, kappa, theta, sigma, rho, T, dt, n_paths, rates_curve = curve_fit):
    """
    Generate synthetic paths for the Heston model.
    
    Parameters:
    - S0: Initial asset price
    - V0: Initial variance
    - mu: Drift term
    - kappa: Mean reversion rate of the variance
    - theta: Long-term variance
    - sigma: Volatility of volatility
    - rho: Correlation between the asset and variance processes
    - T: Time horizon (in years)
    - dt: Time step size
    - n_paths: Number of paths to simulate
    
    Returns:
    - S_paths: Simulated asset price paths (shape: n_paths x n_time_steps)
    - V_paths: Simulated variance paths (shape: n_paths x n_time_steps)
    """
    n_steps = int(T / dt)
    S_paths = np.zeros((n_paths, n_steps + 1))
    V_paths = np.zeros((n_paths, n_steps + 1))
    
    # Initial conditions
    S_paths[:, 0] = S0
    V_paths[:, 0] = V0
    
    # Generate correlated random numbers for the two Wiener processes
    Z1 = np.random.normal(size=(n_paths, n_steps))
    Z2 = np.random.normal(size=(n_paths, n_steps))
    W1 = Z1
    W2 = rho * Z1 + np.sqrt(1 - rho**2) * Z2
    
    # Time stepping through the paths
    for t in range(1, n_steps + 1):
        V_paths[:, t] = V_paths[:, t-1] + kappa * (theta - V_paths[:, t-1]) * dt + \
                        sigma * np.sqrt(np.maximum(V_paths[:, t-1], 0)) * np.sqrt(dt) * W2[:, t-1]
        
        # Ensure variance remains non-negative
        V_paths[:, t] = np.maximum(V_paths[:, t], 0)
        
        S_paths[:, t] = S_paths[:, t-1] * np.exp((mu - 0.5 * V_paths[:, t-1]) * dt + \
                         np.sqrt(V_paths[:, t-1] * dt) * W1[:, t-1])

        
    return S_paths, V_paths

# Example usage
S0 = 5500  # Initial asset price
V0 = 0.04  # Initial variance (volatility squared)
mu = 0.05  # Drift
kappa = 2.0  # Mean reversion rate
theta = 0.04  # Long-term variance
sigma = 0.5  # Volatility of volatility
rho = -0.7  # Correlation between the two Wiener processes
T = 1.0  # 1 year
dt = 1/252  # Daily time steps
n_paths = 1000  # Number of simulated paths

S_paths, V_paths = generate_heston_paths(S0, V0, mu, kappa, theta, sigma, rho, T, dt, n_paths)


In [6]:
#a = 

In [7]:
mu_values = np.arange(0.01,0.11, step = 0.01).round(2).tolist()
kappa_values = np.arange(1.25,3.75, step = 0.25).round(2).tolist()
theta_values = np.arange(0.02,0.12, step = 0.01).round(2).tolist()
sigma_values = np.arange(0.1,1.1, step = 0.1).round(2).tolist()
rho_values = np.arange(-0.1,-1.1, step = -0.1).round(2).tolist()

total_combinations = len(mu_values) * len(kappa_values) * len(theta_values) * len(sigma_values) * len(rho_values)

training_option_prices = []
training_option_params = []


if generate_training_data:

# Iterate over all combinations of parameter sets with a progress bar
    for mu, kappa, theta, sigma, rho in tqdm(itertools.product(mu_values, kappa_values, theta_values, sigma_values, rho_values), total=total_combinations):
        S_paths, V_paths = generate_heston_paths(S0, V0, mu, kappa, theta, sigma, rho, T, dt, n_paths)
        #training_paths.append([mu, kappa, theta, sigma, rho, S_paths[], V_paths])

        temp = np.zeros((n_paths, len(maturities_idx) +1))
        option_prices = np.zeros((len(strikes), len(maturities_idx)))
        for k_idx, k in enumerate(strikes):
            for i, t_idx in enumerate(maturities_idx):
                t = maturities_year[t_idx]
                payoffs = np.maximum(S_paths[:, t_idx] - k, 0)  # Payoff for call option
                temp[:,i ] = payoffs * np.exp(-curve_fit(T- t * dt)* (T - t * dt))
                option_prices[k_idx, :] = temp[:,i].mean(axis = 0)
        #Store or use the generated paths (S_paths) for training your model
        training_option_prices.append(option_prices)
        training_option_params.append([mu, kappa, theta, sigma, rho])

In [8]:
# save copy to save time 
if generate_training_data:
    with open('../equity/training_option_prices.pkl', 'wb') as output:
        pickle.dump(training_option_prices, output)

In [1]:
# load output to save time
if not generate_training_data:
    with open('../equity/training_option_prices.pkl', 'rb') as file:
        training_option_prices = pickle.load(file)

NameError: name 'generate_training_data' is not defined

In [10]:
op_prices = []
op_params = []

for i in training_option_prices:
    op_prices.append(i[5])
    op_params.append(i[:5])

In [11]:
op_prices = torch.Tensor(op_prices)

op_params = torch.Tensor(op_params)

train_size = int(0.8 * len(op_params))  # 80% for training
val_size = len(op_params) - train_size

training_data = TensorDataset(op_params, op_prices)

train_dataset, val_dataset = random_split(training_data, [train_size, val_size])

C:\Users\MITESH\AppData\Local\Temp\ipykernel_36984\2379886255.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:277.)
  op_prices = torch.Tensor(op_prices)


## Model

The deep learning model is a made up of four fully-connected hidden layers each consisting of 30 nodes. The input layer (our Heston model paramters are the inputs) is of size $n$ and output layer is given in the paper as $11$ strikes $\times 8 $ maturities. The paper states the output layer can be edited by the end user. For now we will keep it the same, if it is changed this will be indicated.

In [12]:
class DeepCalibration(nn.Module):
    
    def __init__(self, input_size, output_size):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.fc1 = nn.Sequential(nn.Linear(self.input_size,30), nn.ELU())
        self.fc2 = nn.Sequential(nn.Linear(30,30), nn.ELU())
        self.fc3 = nn.Sequential(nn.Linear(30,30), nn.ELU())
        self.fc4 = nn.Linear(30,self.output_size)

    def forward(self, x):
        
        #assert x.size(-1) == self.input_size

        h1 = self.fc1(x)
        h2 =self.fc2(h1)
        h3 =self.fc3(h2)
        h4 =self.fc4(h3)

        return h4

Initialise models and load to GPU

In [13]:
deep_cal = DeepCalibration(input_size= 5, output_size= 88)

if torch.cuda.is_available():
    deep_cal.to("cuda")  

#summary(deep_cal, )

Create dataloaders to allow efficient data batching for training/validation

In [14]:
# dataloaders

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

val_loader  = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

In [15]:
# train the model

def train_model(dl_model,epochs, training_dl, validation_dl, 
                autograd, dlmodel_loc, trainloss_loc, valloss_loc, trainset_size,valset_size, 
                learning_rate = 0.0001, mse_multp = 1.0, mae_multp = 1.0, return_model = False):
    # define loss and parameters
    
    dl_model.train()

    optimiser = autograd(itertools.chain(dl_model.parameters()), lr=learning_rate)

    train_loss = []
    val_loss = []

    val_loss_check = 1e10

    t0 = time.time()

    print('====Training started====')
    for epoch in range(epochs):
        
        total_mseloss, total_maeloss = 0.0, 0.0
        total_val_loss, total_train_loss = 0.0, 0.0
        val_loss_mse, val_loss_mae = 0.0, 0.0

        for batch_idx, (data, label) in enumerate(training_dl):
            # prepare input data
            img = data.to("cuda")
            lab = label.to("cuda")
            
            model_output = dl_model(img)
            
            loss_mse = mse_loss(model_output.reshape(lab.shape), lab)
            
            loss_mae = l1_loss(model_output.reshape(lab.shape), lab)
        
            loss = (mse_multp * loss_mse) + (mae_multp * loss_mae)
            
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()

            # Track this epoch's training loss
            total_mseloss += loss_mse.item() * data.shape[0] / trainset_size
            total_maeloss += loss_mae.item() * data.shape[0] / trainset_size
            total_train_loss += (loss_mse.item() + loss_mae.item()) * data.shape[0] / trainset_size

        train_loss.append([total_mseloss, total_maeloss , total_train_loss])

        t1 = time.time()

        dl_model.eval()

        with torch.no_grad():
            for batch_idx_val, (data_val, label_val) in enumerate(validation_dl):
                
                img_val = data_val.to("cuda")
                lab_val = label_val.to("cuda")
                
                output_val = dl_model(img_val).reshape(lab_val.shape)
                
                # Track this epoch's validation loss
                val_loss_mse += mse_loss(output_val, lab_val).item() * data_val.shape[0] / valset_size
                val_loss_mae += l1_loss(output_val, lab_val).item() * data_val.shape[0] / valset_size
                total_val_loss += (
                                    (mse_loss(output_val, lab_val).item() * val_loss_mse) +  
                                    (torch.sum(l1_loss(output_val, lab_val)).item()  * mae_multp)
                                  ) * data_val.shape[0] / valset_size

            val_loss.append([val_loss_mse, total_val_loss])

        dl_model.train()

        t2 = time.time()
        #check val error and save model
        if epoch > 0:
            if total_val_loss < val_loss_check:
                torch.save(dl_model, '../' + dlmodel_loc +'intertrain/dlmodel_'+ '_'+ str(epoch) +'_optimal_' + time.strftime("%Y%m%d"))

                if os.path.exists('../' + dlmodel_loc +'intertrain/intertrain_model_log.txt') == False:
                    open('../' + dlmodel_loc +'intertrain/intertrain_model_log.txt', "w").close
                    
                with open('../' + dlmodel_loc +'intertrain/intertrain_model_log.txt', "a") as text_file:
                    text_file.write('====> Epoch: {}, Train time: {:.4f}, Val time: {:.4f} \n'.format(epoch, t1-t0, t2-t1))
                    text_file.write('====> Epoch: {} Total loss: {:.4f} MSE Loss: {:.4f} MAE Loss: {:.4f} Val loss: {:.4f} \n'.format(epoch, total_train_loss, total_mseloss, total_maeloss,total_val_loss))
                    text_file.write('../' + dlmodel_loc +'intertrain/dlmodel_' + '_' + str(epoch) +'_optimal_' + time.strftime("%Y%m%d") + '\n')
                    text_file.write('\n')
                
                val_loss_check = total_val_loss

        if epoch%10==0:  
            print('====> Epoch: {}, Train time: {:.4f}, Val time: {:.4f}'.format(epoch, t1-t0, t2-t1))
            print('====> Epoch: {} Total loss: {:.4f} MSE Loss: {:.4f} MAE Loss: {:.4f} Val loss: {:.4f}'.format(epoch, total_train_loss, total_mseloss, total_maeloss,total_val_loss))
    
    print('====Training complete====')

    torch.save(dl_model, '../' + dlmodel_loc + '_'+ str(epochs) +'_' + time.strftime("%Y%m%d-%H%M%S"))

    train_loss_np = np.array(train_loss)
    val_loss_np = np.array(val_loss)

    with open('../' + trainloss_loc + '_'+ str(epochs) +'_' + time.strftime("%Y%m%d-%H%M%S"), 'wb') as output:
        pickle.dump(train_loss_np, output)

    with open('../' + valloss_loc + '_' + str(epochs) + '_' + time.strftime("%Y%m%d-%H%M%S"), 'wb') as output:
        pickle.dump(val_loss_np, output)

    if return_model:
        return dl_model, train_loss, val_loss

In [16]:
if run_training:

    enc, trainloss,valloss = train_model(
        dl_model= deep_cal,
        epochs = epochs, 
        training_dl = train_loader, 
        validation_dl = val_loader,
        autograd = optim.AdamW, 
        #enc_loc = 'outputs/latent_space_verification/',
        #dec_loc = 'outputs/latent_space_verification/' , 
        dlmodel_loc = 'equity/outputs/final/',
        trainset_size = train_size,
        valset_size = val_size, 
        trainloss_loc = 'equity/outputs/final/train_loss_', 
        valloss_loc = 'equity/outputs/final/val_loss_' , 
        learning_rate= 0.01, mse_multp = 1.0, mae_multp = 0.0, return_model = True
    )

====Training started====
====> Epoch: 0, Train time: 3.3349, Val time: 0.3626
====> Epoch: 0 Total loss: 6.8272 MSE Loss: 5.6200 MAE Loss: 1.2072 Val loss: 3.5525
====> Epoch: 10, Train time: 41.2923, Val time: 0.4178
====> Epoch: 10 Total loss: 3.6000 MSE Loss: 2.6789 MAE Loss: 0.9211 Val loss: 3.6317
====> Epoch: 20, Train time: 80.8503, Val time: 0.4009
====> Epoch: 20 Total loss: 3.5646 MSE Loss: 2.6516 MAE Loss: 0.9130 Val loss: 3.5015
====> Epoch: 30, Train time: 120.6137, Val time: 0.4276
====> Epoch: 30 Total loss: 3.5451 MSE Loss: 2.6347 MAE Loss: 0.9104 Val loss: 4.1065
====> Epoch: 40, Train time: 163.3823, Val time: 0.4458
====> Epoch: 40 Total loss: 3.5518 MSE Loss: 2.6408 MAE Loss: 0.9111 Val loss: 3.1570
====> Epoch: 50, Train time: 207.3175, Val time: 0.4535
====> Epoch: 50 Total loss: 3.5478 MSE Loss: 2.6374 MAE Loss: 0.9104 Val loss: 3.2845
====> Epoch: 60, Train time: 246.8496, Val time: 0.4168
====> Epoch: 60 Total loss: 3.5390 MSE Loss: 2.6300 MAE Loss: 0.9091 Val 

In [17]:
#run DL-model with live prices and compare with numerical calibration